In [1]:
import telebot
import schedule
from time import sleep
import time
from business_models.intranet import DataLens
import os

import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from business_models import greenplum
from business_models.greenplum import GreenplumManager

In [2]:
yoy_sql_wkl = '''
    with source as (

        select date, date - interval '52 weeks' as date_prev, business,
               sum(deliveries) as deliveries,
               sum(gmv) as gmv,
               sum(net_inflow) as net_inflow
        from snb_delivery.dash_bizdev_plan_fact_smb
        where
                scale = 'week'
          and date >= '2022-01-01'
          and country = 'Russia'
          and client_tariff = 'cargo'
          and business in ('C2C','SMB','B2B')
          and plan_name = 'Fact'
        group by 1,2,3
    )

    select
        a.date::date week_dt,
        a.business,
        a.deliveries,
        b.deliveries as deliveries_prev,
        a.gmv,
        b.gmv as gmv_prev,
        a.net_inflow,
        b.net_inflow as net_inflow_prev
    from
        source as a inner join source as b
                               on b.date = a.date_prev
                                   and a.business = b.business
    where a.date >= now()::date - interval '20 weeks'
        and a.date < date_trunc('week', now())
    order by 1,2;
'''

In [3]:
token = '6329128572:AAHwUlRF7aTcF-f3m2CT-lK3jE8aJ0gZk5Y'
chat_id = '-1001944611947'
# chat_id = '-866072730' # cargo core

bot=telebot.TeleBot(token = token)
bot.set_webhook()

True

In [4]:
dataframe = greenplum(yoy_sql_wkl)

In [5]:
metrics = ['deliveries', 'gmv']
businesses = ['total', 'B2B']
scale = 'week_dt'

In [6]:
bot.send_message(chat_id, 'Время недельного отчета!')

i = 0
for b in businesses:

    if b == 'total':
        idf = dataframe.copy()
    else:
        idf = dataframe[dataframe['business']==b]

    idf = idf.groupby([scale]).sum().reset_index()

    for m in metrics:
        idf[f'{m}_yoy'] = (idf[m] - idf[f'{m}_prev']) / idf[f'{m}_prev']

        fig = px.bar(idf, x=scale, y=f'{m}_yoy',
                  template = 'plotly_white',
                  title=f'Cargo_{m}_{b}_yoy')

        fig.add_trace(go.Scatter(
            x=[i for i in idf[scale]],
            y=[i for i in idf[f'{m}_yoy']],
            mode="text",
            name=f"YoY {b} {m}",
            text=[str(round(round(i, 3)*100,1))+'%' for i in idf[f'{m}_yoy']],
            textposition="top center"
        ))
        
        fig.write_image(f"/Users/ghostrider/projects/jupyter_working/kasugai_crow_bot/yoy/fig{i}.png")
        
        pic = open(f'fig{i}.png', 'rb')
        bot.send_photo(chat_id = chat_id, photo = pic)

        i += 1

        #fig.show()

/var/folders/pd/6gyyhtl13jz1jm8hyvvmc0mcsmszvt/T/ipykernel_47235/3957889294.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  idf = idf.groupby([scale]).sum().reset_index()
/var/folders/pd/6gyyhtl13jz1jm8hyvvmc0mcsmszvt/T/ipykernel_47235/3957889294.py:11: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [30]:
f'fig{i}.png'

'fig4.png'

In [22]:
from business_models.botolib import Bot
# cfg = ConfigHolder()
bot_token = token  #достать из mylib_config.json аттрибут test_bot_token
# default_chat_id = -350411529
bot = Bot(token=bot_token, default_chat_id=chat_id)

In [23]:
bot.send_message(message="Hell yes!", chat=chat_id)

In [29]:
bot.send_photo(photo=pic, chat=chat_id)

TypeError: expected str, bytes or os.PathLike object, not BufferedReader

In [25]:
bot.send_photo(chat = chat_id, photo = pic)

TypeError: expected str, bytes or os.PathLike object, not BufferedReader

In [27]:
?bot